# Runge-Kutta Methods

## Derivation and Setup


1. General Runge-Kutta Form

An $s$-stage Runge-Kutta method for solving $y' = f(x,y)$ is defined as:
$$y_{n+1} = y_n + \sum_{i=1}^{s} b_i k_i$$
Where the slopes $k_i$ are:
$$k_i = h f(x_n + c_i h, y_n + \sum_{j=1}^{i-1} a_{ij} k_j)$$

**Parameters to determine:**
* $b_i$: Weights for the final summation.
* $c_i$: Nodes (fraction of the step $h$).
* $a_{ij}$: The Runge-Kutta Matrix (interaction between stages).

---

2. Derivation of RK2 (2nd Order)

**Goal:** Match Taylor Series expansion up to $O(h^2)$.

a. The System of Equations (Order Conditions)
Comparing the Taylor series of the exact solution $y(x+h)$ with the method's expansion yields 3 constraints for 4 unknowns ($b_1, b_2, c_2, a_{21}$):
1.  **Order 1:** $b_1 + b_2 = 1$
2.  **Order 2:** $b_2 c_2 = \frac{1}{2}$
3.  **Order 2:** $b_2 a_{21} = \frac{1}{2}$

*(Note: We typically assume $c_i = \sum a_{ij}$, so $c_2 = a_{21}$, making equation 2 and 3 identical.)*

b. The Choice (Heun's Method)
Since the system is under-determined (3 eqs, 4 unknowns), we make a choice:
* **Choose:** $c_2 = 1$ (Take the slope at the very end of the interval).
* **Result:**
    * From (2): $b_2(1) = 1/2 \implies b_2 = 1/2$.
    * From (1): $b_1 + 1/2 = 1 \implies b_1 = 1/2$.
    * From (3): $a_{21} = c_2 = 1$.

c. Final RK2 Algorithm (Heun)
$$
\begin{aligned}
k_1 &= h f(x_n, y_n) \\
k_2 &= h f(x_n + h, y_n + k_1) \\
y_{n+1} &= y_n + \frac{1}{2}(k_1 + k_2)
\end{aligned}
$$

---

3. Derivation of RK3 (3rd Order)

**Goal:** Match Taylor Series expansion up to $O(h^3)$.

a. The System of Equations
Matching terms up to $h^3$ yields 4 constraints for 6 unknowns ($b_1, b_2, b_3, c_2, c_3, a_{32}$):
1.  $\sum b_i = 1 \implies b_1 + b_2 + b_3 = 1$
2.  $\sum b_i c_i = 1/2 \implies b_2 c_2 + b_3 c_3 = 1/2$
3.  $\sum b_i c_i^2 = 1/3 \implies b_2 c_2^2 + b_3 c_3^2 = 1/3$
4.  $\sum_{ij} b_i a_{ij} c_j = 1/6 \implies b_3 a_{32} c_2 = 1/6$

b. The Choice (Kutta's 3rd Order)
We have 2 degrees of freedom. Standard practice is to space nodes evenly:
* **Choose:** $c_2 = 1/2, c_3 = 1$.
* **Solving:**
    * Substitute $c$'s into (4): $b_3 a_{32} (1/2) = 1/6 \implies b_3 a_{32} = 1/3$.
    * Solve (2) & (3) for weights: Results in $b_1 = 1/6, b_2 = 2/3, b_3 = 1/6$.
    * Solve for matrix: $a_{32} = 2$, which implies $a_{31} = c_3 - a_{32} = 1 - 2 = -1$.

c. Final RK3 Algorithm
$$
\begin{aligned}
k_1 &= h f(x_n, y_n) \\
k_2 &= h f(x_n + \frac{h}{2}, y_n + \frac{k_1}{2}) \\
k_3 &= h f(x_n + h, y_n - k_1 + 2k_2) \\
y_{n+1} &= y_n + \frac{1}{6}(k_1 + 4k_2 + k_3)
\end{aligned}
$$

---

4. Derivation of RK4 (4th Order)

**Goal:** Match Taylor Series expansion up to $O(h^4)$.

a. The System of Equations
Matching terms up to $h^4$ yields 8 constraints for 13 unknowns. The complexity increases due to higher-order derivative terms.
1.  $b_1 + b_2 + b_3 + b_4 = 1$
2.  $b_2 c_2 + b_3 c_3 + b_4 c_4 = 1/2$
3.  $b_2 c_2^2 + b_3 c_3^2 + b_4 c_4^2 = 1/3$
4.  $b_3 a_{32} c_2 + b_4 (a_{42} c_2 + a_{43} c_3) = 1/6$
5.  $b_2 c_2^3 + b_3 c_3^3 + b_4 c_4^3 = 1/4$
6.  $b_3 c_3 a_{32} c_2 + b_4 c_4 (a_{42} c_2 + a_{43} c_3) = 1/8$
7.  $b_3 a_{32} c_2^2 + b_4 (a_{42} c_2^2 + a_{43} c_3^2) = 1/12$
8.  $b_4 a_{43} a_{32} c_2 = 1/24$

b. The Choice (The "Classic" Constraints)
To solve this massive system, Runge and Kutta applied symmetry and simplicity constraints:
* **Symmetry Constraints:** $c_2 = 1/2, c_3 = 1/2, c_4 = 1$.
* **Weight Constraints:** $b_2 = b_3$.
* **Matrix Constraints:** Simple dependence (e.g., $a_{31}=0, a_{41}=0, a_{42}=0$).

**Solving the system with these choices yields:**
* $b_1 = 1/6, b_2 = 1/3, b_3 = 1/3, b_4 = 1/6$.
* $a_{21} = 1/2$.
* $a_{32} = 1/2$.
* $a_{43} = 1$.

c. Final RK4 Algorithm (Classic)
$$
\begin{aligned}
k_1 &= h f(x_n, y_n) \\
k_2 &= h f(x_n + \frac{h}{2}, y_n + \frac{k_1}{2}) \\
k_3 &= h f(x_n + \frac{h}{2}, y_n + \frac{k_2}{2}) \\
k_4 &= h f(x_n + h, y_n + k_3) \\
y_{n+1} &= y_n + \frac{1}{6}(k_1 + 2k_2 + 2k_3 + k_4)
\end{aligned}
$$